# Natural Questions

## Import Statements

In [1]:
import os
import re
import glob
import json

import torch
import jsonlines
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from transformers import BertForQuestionAnswering, TFLongformerForQuestionAnswering
from transformers import BertTokenizer, LongformerTokenizer
import tensorflow as tf

import text_utils

## Read simplified Natural Questions data set (4GB one)

In [2]:
with jsonlines.open('D:\simplified-nq-train.jsonl') as reader:
    i=0
    for line in reader.iter():
        print(line.keys())
        question = line['question_text']
        passage = line['document_text']
        i+=1
        if i>=1:
            break

dict_keys(['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'])


## Data Cleaning

In [3]:
passage_list = BeautifulSoup(passage).find_all('p')
passage_list = [p.text for p in passage_list] 
passage_list = [p.replace('<P>', '') for p in passage_list]
passage_list = [p for p in passage_list if len(p)>2]

In [4]:
# tags = ['<P>', '</P>', '<H1>', '</H1>', '<H2>', '</H2>','<H3>', '</H3>', '<H4>', '</H4>', '<H5>', '</H5>', '<H6>', '</H6>', '<Ul>', '</Ul>', '<Li>', '</Li>']

In [5]:
passage_list = [" ".join([p for p in passage_list[1:]])]

In [6]:
passage_list

[" Email marketing is the act of sending a commercial message , typically to a group of people , using email . In its broadest sense , every email sent to a potential or current customer could be considered email marketing . It usually involves using email to send advertisements , request business , or solicit sales or donations , and is meant to build loyalty , trust , or brand awareness . Marketing emails can be sent to a purchased lead list or a current customer database . The term usually refers to sending email messages with the purpose of enhancing a merchant 's relationship with current or previous customers , encouraging customer loyalty and repeat business , acquiring new customers or convincing current customers to purchase something immediately , and sharing third - party ads .   Email marketing has evolved rapidly alongside the technological growth of the 21st century . Prior to this growth , when emails were novelties to the majority of customers , email marketing was not 

In [7]:
# passage_list = ['Tom Brady holds the records for most wins in nfl with 220']

In [8]:
# passage_list = ["'What Is Love' is a song recorded by the artist Haddaway"]

In [9]:
question

'which is the most common use of opt-in e-mail marketing'

In [10]:
CHUNK_SIZE = 3

## BERT Large Cased SQuAD

In [19]:
# Tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-large-4096-finetuned-triviaqa')

# Model
model = TFLongformerForQuestionAnswering.from_pretrained('allenai/longformer-large-4096-finetuned-triviaqa')

All model checkpoint weights were used when initializing TFLongformerForQuestionAnswering.

All the weights of TFLongformerForQuestionAnswering were initialized from the model checkpoint at allenai/longformer-large-4096-finetuned-triviaqa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerForQuestionAnswering for predictions without further training.


In [20]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
encoding = tokenizer(question, text, return_tensors="pt")
input_ids = encoding["input_ids"]

# default is local attention everywhere
# the forward method will automatically set global attention on question tokens
attention_mask = encoding["attention_mask"]

outputs = model(input_ids, attention_mask=attention_mask)
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

answer_tokens = all_tokens[torch.argmax(start_logits) :torch.argmax(end_logits)+1]
answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens)) # remove space prepending space token

AttributeError: 'torch.Size' object has no attribute 'as_list'